# Business List Quick Formatting

In [46]:
import os
import pandas as pd
import numpy as np
import re

### Read Files and Concatenate
The directory should contain all of the files that will be used for the business list.   
After putting the directory path in as the function's parameter, this will read all of the files and create a single dataframe to work with.

In [47]:
def open_files(path):
    folder = path
    df_list = []
    for file in os.listdir(folder):
        if file.endswith('.csv'):
            file_path=os.path.join(folder,file)
            df = pd.read_csv(file_path)
            df_list.append(df)
    final = pd.concat(df_list, ignore_index=True)
    return final

In [49]:
#df = open_files('/Users/josephmac/Downloads/Project Noble/')
df = open_files('C:/Data/Intermountain Health/')

### Format Data to Fit Template

In [50]:
#Adjust Capitalization
df = df.apply(lambda col:col.str.title() if col.dtypes == 'object' else col)

In [51]:
#Get Formatted Address
cities = pd.read_csv('C:/Data/utah_cities.csv')
cities['Place'] = cities['Place'].str.split(' City').str[0]
cities['Place'] = cities['Place'].str.split(' Town').str[0]
cities['Place'] = cities['Place'].str.split(' Cdp').str[0]
cities_list = cities['Place'].to_list()
for city in cities['Place']:
    mask = df['Address'].str.contains(city, na=False)
    if mask.any():
        # Split only the rows that match
        df.loc[mask, 'street'] = df.loc[mask, 'Address'].str.split(city).str[0].str.strip()
        df.loc[mask, 'city'] = city  
        df['zip_candidates'] = df['Address'].str.findall(r'(\d{5})')
        df['zip'] = df['zip_candidates'].apply(lambda x: x[-1] if x else None)

In [58]:
#Insert NAICS Description
description = pd.read_csv('C:/Data/naics.csv')
description['Code'] = description['Code'].astype(int)
df = df.merge(description[['Code','Name']],left_on='NAICS',right_on='Code',how='left')
df

,Company Name,Patriot Participation,Address,County,City,Phone,Employees,NAICS,street,city,zip_candidates,zip,Code_x,Name_x,Code_y,Name_y,Code,Name
0,Absolute Air Heating & Air Conditio,N,1200 N Hwy 89 Mapleton Ut 84664,Utah,Mapleton,NaN,100-249,238221,1200 N Hwy 89,Mapleton,[84664],84664,NaN,NaN,238221.0,Residential Plumbing and HVAC Contractors,238221,Residential Plumbing and HVAC Contractors
1,Discovery Ranch Inc,N,1308 S 1600 W Mapleton Ut 846644221,Utah,Mapleton,(385) 327-7317,100-249,623220,1308 S 1600 W,Mapleton,[84664],84664,623220.0,Residential Mental Health and Substance Abuse ...,623220.0,Residential Mental Health and Substance Abuse ...,623220,Residential Mental Health and Substance Abuse ...
2,Eckles Paving,N,548 S 700 W Mapleton Ut 846644341,Utah,Mapleton,(801) 377-1190,100-249,237310,548 S 700 W,Mapleton,[84664],84664,237310.0,"Highway, Street, and Bridge Construction",237310.0,"Highway, Street, and Bridge Construction",237310,"Highway, Street, and Bridge Construction"
3,Maple Grove Middle School,N,1587 W 400 S Mapleton Ut 84664,Utah,Mapleton,0,100-249,611110,1587 W 400 S,Mapleton,[84664],84664,611110.0,Elementary and Secondary Schools,611110.0,Elementary and Secondary Schools,611110,Elementary and Secondary Schools
4,Mapleton Junior High,N,362 E 1200 N Mapleton Ut 84664,Utah,Mapleton,0,100-249,611110,362 E 1200 N,Mapleton,[84664],84664,611110.0,Elementary and Secondary Schools,611110.0,Elementary and Secondary Schools,611110,Elementary and Secondary Schools
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,Swd Freight Llc,N,473 S 575 E Santaquin Ut 846558218,Utah,Santaquin,(801) 224-0897,0,488490,473 S 575 E,Santaquin,[84655],84655,488490.0,Other Support Activities for Road Transportation,488490.0,Other Support Activities for Road Transportation,488490,Other Support Activities for Road Transportation
3450,Thumper Jockey Llc,N,806 E 150 S Santaquin Ut 846558265,Utah,Santaquin,(801) 754-5979,0,441330,806 E 150 S,Santaquin,[84655],84655,441330.0,Automotive Parts and Accessories Retailers,441330.0,Automotive Parts and Accessories Retailers,441330,Automotive Parts and Accessories Retailers
3451,Timberline Construction General Con,N,355 S 690 E Santaquin Ut 84655,Utah,Santaquin,(801) 885-2320,0,238111,355 S 690 E,Santaquin,[84655],84655,NaN,NaN,238111.0,Residential Poured Foundation Contractors,238111,Residential Poured Foundation Contractors
3452,Tina Holman,N,214 N 300 E Santaquin Ut 846557114,Utah,Santaquin,(801) 444-3710,0,624120,214 N 300 E,Santaquin,[84655],84655,624120.0,Services for the Elderly and Persons with Disa...,624120.0,Services for the Elderly and Persons with Disa...,624120,Services for the Elderly and Persons with Disa...


### Export File

In [ ]:
#df.to_csv('C:/Data/Business Lists/intermountain_health.csv')